# Cleaning up New York Taxi Cab data
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

Let's use DataPrep to clean and featurize the data which can then be used to predict taxi trip duration. We will not use the For Hire Vehicle (FHV) datasets as they are not really taxi rides and they don't provide drop-off time and geo-coordinates.

In [1]:
from IPython.display import display
from os import path
from tempfile import mkdtemp

import pandas as pd
import azureml.dataprep as dprep

Let's take a quick peek at yellow cab data and green cab data to see what the data looks like. DataPrep supports globing, so you will notice below that we have added a `*` in the path.

*We are using a small sample of the taxi data for this demo. You can find a bigger sample ~6GB by changing the yellow and green dataset's path to:*
*https://dprepdata.blob.core.windows.net/demo/green-sample*<br>
*https://dprepdata.blob.core.windows.net/demo/green-yellow*

In [2]:
pd.set_option('display.max_columns', None)

cache_location = mkdtemp()
dataset_root = "https://dprepdata.blob.core.windows.net/demo"
package_path = path.join(mkdtemp(), "new_york_taxi.dprep")

green_path = "/".join([dataset_root, "green-small/*"])
yellow_path = "/".join([dataset_root, "yellow-small/*"])

print("Retrieving data from the following two sources:")
print(green_path)
print(yellow_path)

green_df = dprep.read_csv(path=green_path, header=dprep.PromoteHeadersMode.GROUPED)
yellow_df = dprep.smart_read_file(path=yellow_path)

display(green_df.head(5))
display(yellow_df.head(5))

Retrieving data from the following two sources:
https://dprepdata.blob.core.windows.net/demo/green-small/*
https://dprepdata.blob.core.windows.net/demo/yellow-small/*


,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,Column21,Column22
0,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,1,0,0,0,0,1,.00,21.25,0,0,0,0,,21.25,2,,,
2,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,1,0,0,0,0,2,.00,74.5,0,0.5,0,0,,75,2,,,
3,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,5,0,0,0,0,1,.00,1,0.1,0,0,1,,2.1,2,,,
4,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,1,0,0,0,0,1,.00,3.25,0,0,0,0,,3.25,2,,,


,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,,,-73.993803,40.695922000000003,CASH,8.9000000000000004,0.5,,0,0,9.4000000000000004
2,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,,,-73.955849999999998,40.768030000000003,Credit,12.1,0.5,,2,0,14.6
3,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,,,-73.869983000000005,40.770225000000003,Credit,23.699999999999999,0,,4.7400000000000002,0,28.440000000000001
4,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,,,-73.996557999999993,40.731848999999997,CREDIT,14.9,0.5,,3.0499999999999998,0,18.449999999999999


## Data Cleanup

Let's define some shortcut transforms that will apply to all Dataflows.

In [3]:
all_columns = dprep.ColumnSelector(term=".*", use_regex=True)
drop_if_all_null = [all_columns, dprep.ColumnRelationship(dprep.ColumnRelationship.ALL)]
useful_columns = [
    "cost", "distance""distance", "dropoff_datetime", "dropoff_latitude", "dropoff_longitude",
    "passengers", "pickup_datetime", "pickup_latitude", "pickup_longitude", "store_forward", "vendor"
]

Let's first work with the green taxi data and get it into a good shape that then can be combined with the yellow taxi data.

In [4]:
tmp_df = (green_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "VendorID": "vendor",
        "lpep_pickup_datetime": "pickup_datetime",
        "Lpep_dropoff_datetime": "dropoff_datetime",
        "lpep_dropoff_datetime": "dropoff_datetime",
        "Store_and_fwd_flag": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Pickup_longitude": "pickup_longitude",
        "Pickup_latitude": "pickup_latitude",
        "Dropoff_longitude": "dropoff_longitude",
        "Dropoff_latitude": "dropoff_latitude",
        "Passenger_count": "passengers",
        "Fare_amount": "cost",
        "Trip_distance": "distance"
     })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,cost
0,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,0,0,0,0,1,21.25
1,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,0,0,0,0,2,74.5
2,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,0,0,0,0,1,1
3,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,0,0,0,0,1,3.25
4,2,2013-08-01 11:51:45,2013-08-01 12:03:52,N,0,0,0,0,1,8.5


In [5]:
green_df = tmp_df

Let's do the same thing to yellow taxi data.

In [6]:
tmp_df = (yellow_df
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "vendor_name": "vendor",
        "VendorID": "vendor",
        "vendor_id": "vendor",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "tpep_pickup_datetime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "store_and_forward": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Passenger_Count": "passengers",
        "passenger_count": "passengers",
        "Fare_Amt": "cost",
        "fare_amount": "cost",
        "Trip_Distance": "distance",
        "trip_distance": "distance"
    })
    .keep_columns(columns=useful_columns))
tmp_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,passengers,pickup_longitude,pickup_latitude,store_forward,dropoff_longitude,dropoff_latitude,cost
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,-73.991956999999999,40.721567,None,-73.993803,40.695922000000003,8.9000000000000004
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,-73.982101999999998,40.736289999999997,None,-73.955849999999998,40.768030000000003,12.1
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,-74.002587000000005,40.739747999999999,None,-73.869983000000005,40.770225000000003,23.699999999999999
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,-73.974266999999998,40.790954999999997,None,-73.996557999999993,40.731848999999997,14.9
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,-74.001580000000004,40.719382000000003,None,-74.008377999999993,40.720350000000003,3.7000000000000002


In [7]:
yellow_df = tmp_df

Let's now append the rows from the `yellow_df` to `green_df`.

In [8]:
combined_df = green_df.append_rows(other_activities=[yellow_df])

Let's take a look at the pickup and drop-off coordinates' data profile to see how the data is distributed.

In [9]:
decimal_type = dprep.TypeConverter(data_type=dprep.FieldType.DECIMAL)
combined_df = combined_df.set_column_types(type_conversions={
    "pickup_longitude": decimal_type,
    "pickup_latitude": decimal_type,
    "dropoff_longitude": decimal_type,
    "dropoff_latitude": decimal_type
})
combined_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Error Count,Lower Quartile,Upper Quartile,Standard Deviation,Mean
pickup_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,0.0,-73.947693,-73.846670,18.792672,-68.833579
pickup_latitude,FieldType.DECIMAL,0.000000,40.919121,7722.0,0.0,0.0,40.721075,40.803909,10.345967,37.936742
dropoff_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,0.0,-73.956250,-73.866208,18.696526,-68.896978
dropoff_latitude,FieldType.DECIMAL,0.000000,41.008934,7722.0,0.0,0.0,40.717821,40.784688,10.290780,37.963774


From the data profile, we can see that there are coordinates that are missing and coordinates that are not in New York. Let's filter out coordinates not in the [city border](https://mapmakerapp.com?map=5b60a055a191245990310739f658).

In [10]:
tmp_df = (combined_df
    .drop_nulls(
        columns=["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
        column_relationship=dprep.ColumnRelationship(dprep.ColumnRelationship.ANY)
    ) 
    .filter(dprep.f_and(
        dprep.col("pickup_longitude") <= -73.72,
        dprep.col("pickup_longitude") >= -74.09,
        dprep.col("pickup_latitude") <= 40.88,
        dprep.col("pickup_latitude") >= 40.53,
        dprep.col("dropoff_longitude") <= -73.72,
        dprep.col("dropoff_longitude") >= -74.09,
        dprep.col("dropoff_latitude") <= 40.88,
        dprep.col("dropoff_latitude") >= 40.53
    )))
tmp_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude", 
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Error Count,Lower Quartile,Upper Quartile,Standard Deviation,Mean
pickup_longitude,FieldType.DECIMAL,-74.078156,-73.736481,7059.0,0.0,0.0,-73.948975,-73.866662,0.048711,-73.913865
pickup_latitude,FieldType.DECIMAL,40.575485,40.879852,7059.0,0.0,0.0,40.721403,40.805145,0.048348,40.765226
dropoff_longitude,FieldType.DECIMAL,-74.085747,-73.720871,7059.0,0.0,0.0,-73.959041,-73.884846,0.055961,-73.920718
dropoff_latitude,FieldType.DECIMAL,40.583530,40.879734,7059.0,0.0,0.0,40.727549,40.788378,0.050462,40.759487


In [11]:
combined_df = tmp_df

Let's take a look at the data profile for the `store_forward` column.

In [12]:
combined_df.keep_columns(columns='store_forward').get_profile()

,Type,Min,Max,Count,Missing Count,Error Count,Lower Quartile,Upper Quartile,Standard Deviation,Mean
store_forward,FieldType.STRING,N,Y,7059.0,99.0,0.0,,,,


From the data profile of `store_forward` above, we can see that the data is inconsistent and there are missing values. Let's fix them.

In [13]:
combined_df = combined_df.replace(columns="store_forward", find="0", replace_with="N").fill_nulls("store_forward", "N")

Let's now split the pick up and drop off datetimes into a date column and a time column. We will use `split_column_by_example` to perform the split. If the `example` parameter of `split_column_by_example` is omitted, we will automatically try to figure out where to split based on the data.

In [14]:
tmp_df = (combined_df
    .split_column_by_example(source_column="pickup_datetime")
    .split_column_by_example(source_column="dropoff_datetime"))
tmp_df.head(5)

,vendor,pickup_datetime,pickup_datetime_1,pickup_datetime_2,dropoff_datetime,dropoff_datetime_1,dropoff_datetime_2,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,3.3


In [15]:
combined_df = tmp_df

Let's rename the columns generated by `split_column_by_example` into meaningful names.

In [16]:
tmp_df = (combined_df
    .rename_columns(column_pairs={
        "pickup_datetime_1": "pickup_date",
        "pickup_datetime_2": "pickup_time",
        "dropoff_datetime_1": "dropoff_date",
        "dropoff_datetime_2": "dropoff_time"
    }))
tmp_df.head(5)

,vendor,pickup_datetime,pickup_date,pickup_time,dropoff_datetime,dropoff_date,dropoff_time,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,3.3


In [17]:
combined_df = tmp_df

## Feature Engineering

### Datetime features

Let's split the pickup and drop-off date further into day of week, day of month, and month. For pickup and drop-off time columns, we will split it into hour, minute, and second.

In [18]:
tmp_df = (combined_df
    .derive_column_by_example(
        source_columns="pickup_date", 
        new_column_name="pickup_weekday", 
        example_data=[("2009-01-04", "Sunday"), ("2013-08-22", "Thursday")]
    )
    .derive_column_by_example(
        source_columns="dropoff_date",
        new_column_name="dropoff_weekday",
        example_data=[("2013-08-22", "Thursday"), ("2013-11-03", "Sunday")]
    )
    .split_column_by_example(source_column="pickup_date")
    .split_column_by_example(source_column="pickup_time")
    .split_column_by_example(source_column="dropoff_date")
    .split_column_by_example(source_column="dropoff_time")
    .split_column_by_example(source_column="pickup_time_1")
    .split_column_by_example(source_column="dropoff_time_1")
    .drop_columns(columns=[
        "pickup_date", "pickup_time", "dropoff_date", "dropoff_time", 
        "pickup_date_1", "dropoff_date_1", "pickup_time_1", "dropoff_time_1"
    ])
    .rename_columns(column_pairs={
        "pickup_date_2": "pickup_month",
        "pickup_date_3": "pickup_monthday",
        "pickup_time_1_1": "pickup_hour",
        "pickup_time_1_2": "pickup_minute",
        "pickup_time_2": "pickup_second",
        "dropoff_date_2": "dropoff_month",
        "dropoff_date_3": "dropoff_monthday",
        "dropoff_time_1_1": "dropoff_hour",
        "dropoff_time_1_2": "dropoff_minute",
        "dropoff_time_2": "dropoff_second"
    }))
tmp_df.head(5)

,vendor,pickup_datetime,pickup_month,pickup_monthday,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_datetime,dropoff_month,dropoff_monthday,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,cost
0,2,2013-08-01 17:22:00,08,01,Thursday,17,22,00,2013-08-01 17:22:00,08,01,Thursday,17,22,00,N,-73.937767,40.758480,-73.937767,40.758480,1,2.5
1,2,2013-08-01 17:24:00,08,01,Thursday,17,24,00,2013-08-01 17:25:00,08,01,Thursday,17,25,00,N,-73.937927,40.757843,-73.937927,40.757843,1,2.5
2,2,2013-08-06 06:51:19,08,06,Tuesday,06,51,19,2013-08-06 06:51:36,08,06,Tuesday,06,51,36,N,-73.937721,40.758404,-73.937721,40.758369,1,3.3
3,2,2013-08-06 13:26:34,08,06,Tuesday,13,26,34,2013-08-06 13:26:57,08,06,Tuesday,13,26,57,N,-73.937691,40.758419,-73.937790,40.758358,1,3.3
4,2,2013-08-06 13:27:53,08,06,Tuesday,13,27,53,2013-08-06 13:28:08,08,06,Tuesday,13,28,08,N,-73.937805,40.758396,-73.937775,40.758450,1,3.3


In [19]:
combined_df = tmp_df

From the data above, we can see that the pickup and drop-off date and time components produced from the transforms above looks good. Let's drop the `pickup_datetime` and `dropoff_datetime` columns as they are no longer needed.

In [20]:
tmp_df = combined_df.drop_columns(columns=["pickup_datetime", "dropoff_datetime"])
tmp_df.head(5)

,vendor,pickup_month,pickup_monthday,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_month,dropoff_monthday,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,cost
0,2,08,01,Thursday,17,22,00,08,01,Thursday,17,22,00,N,-73.937767,40.758480,-73.937767,40.758480,1,2.5
1,2,08,01,Thursday,17,24,00,08,01,Thursday,17,25,00,N,-73.937927,40.757843,-73.937927,40.757843,1,2.5
2,2,08,06,Tuesday,06,51,19,08,06,Tuesday,06,51,36,N,-73.937721,40.758404,-73.937721,40.758369,1,3.3
3,2,08,06,Tuesday,13,26,34,08,06,Tuesday,13,26,57,N,-73.937691,40.758419,-73.937790,40.758358,1,3.3
4,2,08,06,Tuesday,13,27,53,08,06,Tuesday,13,28,08,N,-73.937805,40.758396,-73.937775,40.758450,1,3.3


In [21]:
combined_df = tmp_df

Let's now save the transformation steps into a DataPrep package so we can use it to to run on spark.

In [22]:
combined_df = combined_df.set_name(name="nyc_taxi")
package = dprep.Package(arg=combined_df)
package = package.save(file_path=package_path)